In [1]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

2023-10-04 00:18:02.452635: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-04 00:18:02.482079: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-04 00:18:02.482661: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 00:18:03.237518: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def read_csv_files_from_folder(folder_path):
    # Initialize an empty list to store DataFrames
    dfs = []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files in the folder
    for file_name in file_list:
        # Check if the file has a .csv extension
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)

            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Append the DataFrame to the list
            dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [3]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [4]:
results = read_csv_files_from_folder('results/')
results.replace({'ftl_mlp_initializer': 'Fair Transition Loss', 'adversarial_debiasing_initializer': 'Adversarial Debiasing', 'gerry_fair_classifier_initializer': 'Gerry Fair Classifier', 'prejudice_remover_initializer': 'Prejudice Remover', 'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)
results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [5]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

results['Performance'] = 0
results['Fairness'] = 0
results['Fitness Rule'] = ''
for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
    results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

In [6]:
display(results)

,Unnamed: 0,dataset,method,fitness_rule,fitness,Accuracy,Mathew Correlation,Equalized Odds,Statistical Parity,Equal Opportunity,Performance,Fairness,Fitness Rule,Fitness Rule Abvr
0,3,German Credit,Prejudice Remover,mcc_parity,-0.077859,0.545000,0.000000,0.070884,0.077859,0.056402,0.000000,0.077859,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
1,7,German Credit,Prejudice Remover,mcc_odds,-0.129086,0.620000,0.000000,0.129086,0.161000,0.084427,0.000000,0.129086,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
2,11,German Credit,Prejudice Remover,mcc_opportunity,-0.053052,0.575000,0.000000,0.115271,0.135058,0.053052,0.000000,0.053052,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
3,15,German Credit,Prejudice Remover,acc_parity,0.392316,0.555000,0.000000,0.159695,0.162684,0.195652,0.555000,0.162684,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
4,19,German Credit,Prejudice Remover,acc_odds,0.469445,0.555000,0.000000,0.085555,0.021354,0.117521,0.555000,0.085555,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1734,Bank Marketing,Fair Transition Loss,mcc_odds,0.512368,0.892752,0.567577,0.055209,0.064039,0.096869,0.567577,0.055209,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
1841,1739,Bank Marketing,Fair Transition Loss,mcc_opportunity,0.439395,0.787963,0.501721,0.423663,0.696893,0.062327,0.501721,0.062327,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
1842,1744,Bank Marketing,Fair Transition Loss,acc_parity,0.885779,0.893244,0.378632,0.047285,0.007465,0.079024,0.893244,0.007465,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
1843,1749,Bank Marketing,Fair Transition Loss,acc_odds,0.823758,0.895048,0.393785,0.071289,0.012262,0.137643,0.895048,0.071289,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)


In [9]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [10]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [11]:
methods = ['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier']
methods

['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier']

In [12]:
if os.path.exists('multi_aso_data_list.json'):
    with open('multi_aso_data_list.json') as file:
        multi_aso_data_list = json.load(file)
else:    
    multi_aso_data_list = []
    for d in datasets:
        multi_aso_data = []
        for f in fitness_rules:
            methods_results = []
            for m in methods:
                r = results.loc[ (results['dataset'] == d) &
                                     (results['fitness_rule'] == f) &
                                     (results['method'] == m) ]\
                            .fitness.tolist()
                if len(r) == 0:
                    r = [-1]
                methods_results.append(r)
            min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
            multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
    with open('multi_aso_data_list.json', 'w') as file:
        json.dump(multi_aso_data_list, file)

Model comparisons: 100%|█████████▉| 9985/10000 [00:50<00:00, 198.71it/s]

Model comparisons:   0%|          | 0/10000 [00:00<?, ?it/s]

Model comparisons:   0%|          | 19/10000 [00:00<00:54, 182.11it/s]

Model comparisons:   0%|          | 38/10000 [00:00<00:54, 184.47it/s]

Model comparisons:   1%|          | 58/10000 [00:00<00:52, 188.89it/s]

Model comparisons:   1%|          | 78/10000 [00:00<00:52, 189.60it/s]

Model comparisons:   1%|          | 97/10000 [00:00<00:52, 189.03it/s]

Model comparisons:   1%|          | 118/10000 [00:00<00:51, 193.67it/s]

Model comparisons:   1%|▏         | 138/10000 [00:00<00:50, 195.40it/s]

Model comparisons:   2%|▏         | 159/10000 [00:00<00:49, 198.09it/s]

Model comparisons:   2%|▏         | 179/10000 [00:00<00:49, 198.59it/s]

Model comparisons:   2%|▏         | 200/10000 [00:01<00:49, 198.93it/s]

Model comparisons:   2%|▏         | 220/10000 [00:01<00:49, 198.58it/s]

Model comparisons:   2%|▏         | 240/10000 [00:01<00:49, 198.90

In [13]:
aso_df_resume = []
for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
    fitness_rule = aso_result['fitness_rule']
    dataset = aso_result['dataset']

    aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
    aso_df['method'] = methods
    aso_df['dataset'] = dataset
    aso_df['fitness_rule'] = fitness_rule
    aso_df = aso_df[aso_df['method'] == 'Fair Transition Loss' ]
    aso_df = aso_df.drop(['Fair Transition Loss'], axis=1)
    aso_df = aso_df.drop(['method'], axis=1)
    aso_df_resume.append(aso_df)


print('Significance Testing')
significance = pd.concat(aso_df_resume)
significance = significance.set_index(['fitness_rule', 'dataset'])
significance = significance.sort_values(by=['fitness_rule', 'dataset'])
display(significance)
significance.to_latex('tables/significance_resume.tex')

Significance Testing


Standard MLP (baseline)  \
fitness_rule    dataset                                      
acc_odds        Adult Income                      0.008806   
                Bank Marketing                    0.768386   
                Compas Recidivism                 0.006591   
                German Credit                     1.000000   
acc_opportunity Adult Income                      0.889891   
                Bank Marketing                    1.000000   
                Compas Recidivism                 0.009003   
                German Credit                     0.013958   
acc_parity      Adult Income                      0.006234   
                Bank Marketing                    0.253591   
                Compas Recidivism                 0.002049   
                German Credit                     1.000000   
mcc_odds        Adult Income                      0.025439   
                Bank Marketing                    0.458281   
                Compas Recidivism                 0.005348   
                German Credit                     1.000000   
mcc_opportunity Adult Income                      0.008600   
                Bank Marketing                    0.806930   
                Compas Recidivism                 0.004847   
                German Credit                     1.000000   
mcc_parity      Adult Income                      0.004635   
                Bank Marketing                    0.007924   
                Compas Recidivism                 0.005117   
                German Credit                     0.366452   

                                   Adversarial Debiasing  Prejudice Remover  \
fitness_rule    dataset                                                       
acc_odds        Adult Income                    0.224148           0.189303   
                Bank Marketing                  0.406367           0.808664   
                Compas Recidivism               0.462377           0.000000   
                German Credit                   0.426571           0.957387   
acc_opportunity Adult Income                    0.965546           1.000000   
                Bank Marketing                  0.389006           0.810667   
                Compas Recidivism               0.788252           0.000000   
                German Credit                   0.063457           0.000000   
acc_parity      Adult Income                    0.239690           0.303426   
                Bank Marketing                  1.000000           1.000000   
                Compas Recidivism               1.000000           0.100424   
                German Credit                   0.472299           1.000000   
mcc_odds        Adult Income                    0.260956           0.444356   
                Bank Marketing                  0.189010           0.116477   
                Compas Recidivism               0.572783           0.000000   
                German Credit                   1.000000           1.000000   
mcc_opportunity Adult Income                    0.001726           0.051493   
                Bank Marketing                  0.177162           0.250333   
                Compas Recidivism               1.000000           0.000000   
                German Credit                   1.000000           1.000000   
mcc_parity      Adult Income                    0.160518           1.000000   
                Bank Marketing                  0.000000           0.000000   
                Compas Recidivism               0.260848           0.000000   
                German Credit                   1.000000           0.500420   

                                   Gerry Fair Classifier  
fitness_rule    dataset                                   
acc_odds        Adult Income                    0.000000  
                Bank Marketing                  1.000000  
                Compas Recidivism               0.005790  
                German Credit                   0.556245  
acc_opportuni

In [14]:
grouped_results = results\
    .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
    .agg({'fitness': ['mean', 'std'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
    .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
grouped_results

fitness  \
                                                                   mean   
Fitness Rule Abvr       dataset       method                              
Max(MCC - Stat. Parity) German Credit Adversarial Debiasing    0.200070   
                                      Gerry Fair Classifier   -0.031208   
                                      Fair Transition Loss    -0.034435   
                                      Prejudice Remover       -0.082559   
                                      Standard MLP (baseline) -0.090370   
...                                                                 ...   
Max(Acc - Eq. Odds)     Adult Income  Prejudice Remover        0.809534   
                                      Fair Transition Loss     0.787013   
                                      Adversarial Debiasing    0.756054   
                                      Standard MLP (baseline)  0.752260   
                                      Gerry Fair Classifier    0.704668   

                                                                         \
                                                                    std   
Fitness Rule Abvr       dataset       method                              
Max(MCC - Stat. Parity) German Credit Adversarial Debiasing    0.172675   
                                      Gerry Fair Classifier    0.052788   
                                      Fair Transition Loss     0.073668   
                                      Prejudice Remover        0.050746   
                                      Standard MLP (baseline)  0.061469   
...                                                                 ...   
Max(Acc - Eq. Odds)     Adult Income  Prejudice Remover        0.022701   
                                      Fair Transition Loss     0.083934   
                                      Adversarial Debiasing    0.028345   
                                      Standard MLP (baseline)  0.041790   
                                      Gerry Fair Classifier    0.074245   

                                                              Performance  \
                                                                     mean   
Fitness Rule Abvr       dataset       method                                
Max(MCC - Stat. Parity) German Credit Adversarial Debiasing      0.367673   
                                      Gerry Fair Classifier      0.000000   
                                      Fair Transition Loss       0.000000   
                                      Prejudice Remover          0.000000   
                                      Standard MLP (baseline)    0.000000   
...                                                                   ...   
Max(Acc - Eq. Odds)     Adult Income  Prejudice Remover          0.845788   
                                      Fair Transition Loss       0.825826   
                                      Adversarial Debiasing      0.847730   
                                      Standard MLP (baseline)    0.849309   
                                      Gerry Fair Classifier      0.750809   

                                                                         \
                                                                    std   
Fitness Rule Abvr       dataset       method                              
Max(MCC - Stat. Parity) German Credit Adversarial Debiasing    0.056485   
                                      Gerry Fair Classifier    0.000000   
                                      Fair Transition Loss     0.000000   
                                      Prejudice Remover        0.000000   
                                      Standard MLP (baseline)  0.000000   
...                                                                 ...   
Max(Acc - Eq. Odds)     Adult Income  Prejudice Remover        0.003691   
                                      Fair Transition Loss     0.069484   
                                      Adversarial Debiasin

In [15]:
selected_columns = ['formatted_fitness', 'formatted_performance', 'formatted_fairness']
grouped_results[selected_columns].to_latex('tables/grouped_results.tex')